In [ ]:
import cv2
import matplotlib.pyplot as plt
import numpy as np
import time, os

#### Initialise network

In [ ]:
#net = cv2.dnn.readNet('yolov2-tiny.weights', 'tinyYolo.cfg')
#net = cv2.dnn.readNet('yolov3.weights', 'yolo_v3_416.cfg')
#net = cv2.dnn.readNet('yolov3_608.weights', 'yolo_v3_608.cfg')
net = cv2.dnn.readNet('yolov3.weights','yolov3.cfg')

with open("coco.names", "r") as f:
    classes = [line.strip() for line in f.readlines()]

layer_names = net.getLayerNames()
output_layers = [layer_names[i[0] - 1] for i in net.getUnconnectedOutLayers()]
colors = np.random.uniform(0, 255, size=(len(classes), 3))

img = cv2.imread('sequence/000001.jpg')
img.shape
height, width, channels = img.shape

blob = cv2.dnn.blobFromImage(img, 0.00392, (416, 416), (0, 0, 0), True, crop=False)
net.setInput(blob)
outs = net.forward(output_layers)

#### Run detection on video

In [ ]:
def get_detection(img):
    blob = cv2.dnn.blobFromImage(img, 0.00392, (416, 416), (0, 0, 0), True, crop=False)
    net.setInput(blob)
    outs = net.forward(output_layers)
    
    class_ids = []
    confidences = []
    boxes = []
    for out in outs:
        for detection in out:
            scores = detection[5:]
            class_id = np.argmax(scores)
            confidence = scores[class_id]
            if confidence > 0.5:
                # Object detected
                center_x = int(detection[0] * width)
                center_y = int(detection[1] * height)
                w = int(detection[2] * width)
                h = int(detection[3] * height)
                # Rectangle coordinates
                x = int(center_x - w / 2)
                y = int(center_y - h / 2)
                boxes.append([x, y, w, h])
                confidences.append(float(confidence))
                class_ids.append(class_id)
    indexes = cv2.dnn.NMSBoxes(boxes, confidences, 0.5, 0.4)
    return boxes, indexes, class_ids


def draw_rectangles(boxes, indexes, class_ids):
    font = cv2.FONT_HERSHEY_PLAIN
    for i in range(len(boxes)):
        if i in indexes:
            x, y, w, h = boxes[i]

            label = str(classes[class_ids[i]])
            if label == 'person':
                color = colors[i]
                cv2.rectangle(img, (x, y), (x + w, y + h), (255,0,0), 2)
                #cv2.putText(img, label, (x, y + 30), font, 3, (255,0,0), 3)

    cv2.imshow("Image", img)
    cv2.waitKey(1)   
    return img


#### Playback

In [ ]:
nb_frames = 10

for i in range(1, nb_frames):
    path = f'sequence/{i:06d}.jpg'
    img = cv2.imread(path)
    boxes, indexes, class_ids = get_detection(img)
    draw_rectangles(boxes, indexes, class_ids)

cv2.destroyAllWindows()

#### Writing to files

In [ ]:
nb_pictures = 10#795

for i in range(1, nb_pictures):
    path = f'sequence/{i:06d}.jpg'
    img = cv2.imread(path)
    boxes, indexes, class_ids = get_detection(img)
#     print("**********Boxes:")
#     print(boxes)
#     print("**********Indexes:")
#     print(indexes)
#     print('')
    img = draw_rectangles(boxes, indexes, class_ids)
    cv2.imwrite(f'render/rect.{i:06d}.jpg', img)


cv2.destroyAllWindows()